# Home Work 8.

### Снижение размерности данных

Для выполнения домашнего задания будем использовать файл Lesson_8_extended.ipynb (из материалов к уроку):

### Задача 1.  
**Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.**

In [1]:
import numpy as np

from sklearn import model_selection
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [2]:
# Загрузим игрушечный датасет из sklearn
X, y = load_iris(return_X_y=True)

In [3]:
# Для начала отмасштабируем выборку

def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [4]:
X = X.astype(float)

X_scaled = standard_scale(X)

In [5]:
# Разобьём выборку на обучающую и тестовую.
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.3, random_state=42)

Воспользуемся написанным на прошлом уроке алгоритмом KNN.

In [6]:
"""Евклидова метрика расстояния"""
def e_metrics(x1, x2):
    
    distance = np.sum(np.square(x1 - x2))

    return np.sqrt(distance)

In [7]:
"""Вычисление веса от расстояния"""
weight = lambda d: 0.5 ** d

In [8]:
"""Метрика точности"""
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [9]:
"""Алгоритм поиска K ближайших соседей"""
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем суммарные веса разных классов
        for distance, answer in sorted(test_distances)[0:k]:
            classes[answer] += weight(distance)

        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [10]:
k = 2
y_test_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_test_pred, y_test):.3f}')

Точность алгоритма при k = 2: 0.978


Точность алгоритма на тестовой выборке довольно большая, даже для k=2. Реализуем алгоритм PCA.

In [11]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_scaled.T @ X_scaled

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979894, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021055, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569725, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.1072254642929513, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [12]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [13]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_scaled.dot(W)

In [14]:
# Новое разбиение на обучающую тестовую выборки.
X_train, X_test, y_train, y_test = model_selection.train_test_split(Z, y, test_size=0.3, random_state=42)

In [15]:
k = 2
y_test_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_test_pred, y_test):.3f}')

Точность алгоритма при k = 2: 0.889


### Задача 2.  
**Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()**

In [16]:
"""Алгоритм понижения размерности на основе сингулярного разложения"""
def my_pca_svd(X, d):
    u, s, vh = np.linalg.svd(X, full_matrices=False)
    W = vh[:, :d]
    return X @ W

In [17]:
# Новая матрица признаков.
Z = my_pca_svd(X_scaled, 2)

# Новое разбиение на обучающую тестовую выборки.
X_train, X_test, y_train, y_test = model_selection.train_test_split(Z, y, test_size=0.3, random_state=42)

In [18]:
k = 2
y_test_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_test_pred, y_test):.3f}')

Точность алгоритма при k = 2: 0.867
